# Make dictionary

In [1]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSequenceClassification, 
    DataCollatorForTokenClassification, 
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer, 
    DataCollatorForLanguageModeling
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from transformers.models.bert.configuration_bert import BertConfig 
import numpy as np
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
#from training_args_module import training_args
import transformers
import os

import csv
from Bio import SeqIO


import argparse

/home/aa8417/.conda/envs/dna/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
princeton_id = 'aa8417'
project_dir = f'/scratch/gpfs/{princeton_id}/QCB557_project'

model_name = 'fine_tune_new_v3'
model_out_dir = f'{project_dir}/models/{model_name}'

# use gpu
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
config = BertConfig.from_pretrained(f'{model_out_dir}/config.json', output_attentions=True)
print(config.num_labels) #2 labels
model_base = AutoModel.from_pretrained(model_out_dir, trust_remote_code=True, config=config)
model_base.to(device)

2


Some weights of the model checkpoint at /scratch/gpfs/aa8417/QCB557_project/models/fine_tune_new_v3 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(4096, 768, padding_idx=0)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertUnpadAttention(
          (self): BertUnpadSelfAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (mlp): BertGatedLinearUnitMLP(
          (gated_layers): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELU(approximate='none')
          

In [4]:
config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M", output_hidden_states=True, output_attentions=True)
print(config.num_labels) #2 labels
model_seq = AutoModelForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
model_seq.to(device)

2


Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly ini

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4096, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertUnpadAttention(
            (self): BertUnpadSelfAttention(
              (dropout): Dropout(p=0.0, inplace=False)
              (Wqkv): Linear(in_features=768, out_features=2304, bias=True)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (mlp): BertGatedLinearUnitMLP(
            (gated_layers): Linear(in_features=768

In [5]:
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, padding=True)
tokenizer.pad_token = "X"

In [8]:
test = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/H3K4me3/test.csv')
train = pd.read_csv('/scratch/gpfs/aa8417/QCB557_project/data/H3K4me3/train.csv')
full = pd.concat([train, test], ignore_index=True)

In [9]:
full

,sequence,label
0,ACTACAACTGTTTAGCTACTAACTTGCTTGACATCCAAAGAGCAGG...,1
1,GTTACCTCCTTACTGTGTATGCATCAGGTATGCATTTTGTGATATA...,1
2,CCCTCATTAAAGCAACTTATGCACAAGAGGGACTAAAGGGATTTTA...,0
3,GGTTGAACGCTGGCTGCAGTACCTGAACAACGAGAAAAAGAAAACG...,1
4,TTCGCGAAGGATGCAACTAAGTAGAGCAAAAGCTGAAAACTCCAAA...,0
...,...,...
36794,ATCATTCGGACAATTATTGGCATTCAAGAAATTGAATAAATCCGCG...,0
36795,TTCAAGACATATTAGCACTGGTGCCATATTGAATAACACAATCGCG...,0
36796,TAAAACGGGTCCACAATATAAGGTCAATAACATCGCAACACATTTC...,1
36797,CGTACAGCGAAAAAACATTAGTAACTTTATTTCCTTATTTCAGGGC...,0


In [ ]:
def get_model_output(model_base, model_seq, tokenizer, dataframe, device):
    results_dict = {}

    for index, row in dataframe.iterrows():
        sequence = row['sequence']
        label = row['label']
        
        inputs = tokenizer(sequence, padding='max_length', max_length=128, return_tensors='pt').to(device)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            outputs_seq = model_seq(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
            attention_weights = outputs_seq.attentions
            hidden_states = outputs_seq.hidden_states
        
        with torch.no_grad()
            outputs_base = model_base(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
            attention_weights = outputs_base.attentions

        key = f'seq_{index}_{label}'
        results_dict[key] = {
            "sequence": sequence,
            "input_ids": input_ids.tolist(),
            "hidden_states": hidden_states,
            "attention_weights": attention_weights
        }

    return results_dict

In [ ]:
results_dict = get_model_output(model_base, model_seq, tokenizer, data, device)

json_file_path = '/scratch/gpfs/aa8417/QCB557_project/data/H3K4me3/results_dict.json'

with open(json_file_path, 'w') as json_file:
    json.dump(results_dict, json_file)